## Building A Chatbot
In this video We'll go over an example of how to design and implement an LLM-powered chatbot. This chatbot will be able to have a conversation and remember previous interactions.

Note that this chatbot that we build will only use the language model to have a conversation. There are several other related concepts that you may be looking for:

- Conversational RAG: Enable a chatbot experience over an external source of data
- Agents: Build a chatbot that can take actions

This video tutorial will cover the basics which will be helpful for those two more advanced topics.

In [30]:
import os
from dotenv import load_dotenv
load_dotenv() ## aloading all the environment variable

groq_api_key=os.getenv("GROQ_API_KEY")



In [2]:
from langchain_groq import ChatGroq
import httpx

custom_http_client = httpx.Client(verify=False)
model=ChatGroq(model="Gemma2-9b-It",groq_api_key=groq_api_key,http_client=custom_http_client)
model

ChatGroq(client=<groq.resources.chat.completions.Completions object at 0x0000013CE78E8350>, async_client=<groq.resources.chat.completions.AsyncCompletions object at 0x0000013CE7C9B990>, model_name='Gemma2-9b-It', http_client=<httpx.Client object at 0x0000013CE7A0F010>)

In [3]:
from langchain_core.messages import HumanMessage
model.invoke([HumanMessage(content="Hi , My name is Krish and I am a Chief AI Engineer")])

AIMessage(content="Hello Krish, it's nice to meet you!\n\nBeing a Chief AI Engineer is a fascinating role. What kind of AI projects are you currently working on?  \n\nI'm eager to learn more about your work and how I can be helpful.\n", response_metadata={'token_usage': {'completion_tokens': 55, 'prompt_tokens': 22, 'total_tokens': 77, 'completion_time': 0.1, 'prompt_time': 0.002126796, 'queue_time': 0.245224204, 'total_time': 0.102126796}, 'model_name': 'Gemma2-9b-It', 'system_fingerprint': 'fp_10c08bf97d', 'finish_reason': 'stop', 'logprobs': None}, id='run-66e00942-8196-4b4c-834e-e4156eca5682-0', usage_metadata={'input_tokens': 22, 'output_tokens': 55, 'total_tokens': 77})

In [4]:
from langchain_core.messages import AIMessage
model.invoke(
    [
        HumanMessage(content="Hi , My name is Krish and I am a Chief AI Engineer"),
        AIMessage(content="Hello Krish! It's nice to meet you. \n\nAs a Chief AI Engineer, what kind of projects are you working on these days? \n\nI'm always eager to learn more about the exciting work being done in the field of AI.\n"),
        HumanMessage(content="Hey What's my name and what do I do?")
    ]
)

AIMessage(content="You are Krish, and you are a Chief AI Engineer! \n\nIs there anything else you'd like me to remember about you?  I'm here to help in any way I can. 😊 \n", response_metadata={'token_usage': {'completion_tokens': 46, 'prompt_tokens': 97, 'total_tokens': 143, 'completion_time': 0.083636364, 'prompt_time': 0.008938704, 'queue_time': 0.815247426, 'total_time': 0.092575068}, 'model_name': 'Gemma2-9b-It', 'system_fingerprint': 'fp_10c08bf97d', 'finish_reason': 'stop', 'logprobs': None}, id='run-57235971-f97d-40bf-a4b4-9369f7c1629c-0', usage_metadata={'input_tokens': 97, 'output_tokens': 46, 'total_tokens': 143})

### Message History
We can use a Message History class to wrap our model and make it stateful. This will keep track of inputs and outputs of the model, and store them in some datastore. Future interactions will then load those messages and pass them into the chain as part of the input. Let's see how to use this!

In [5]:
from langchain_community.chat_message_histories import ChatMessageHistory
from langchain_core.chat_history import BaseChatMessageHistory
from langchain_core.runnables.history import RunnableWithMessageHistory

store={}

def get_session_history(session_id:str)->BaseChatMessageHistory:
    if session_id not in store:
        store[session_id]=ChatMessageHistory()
    return store[session_id]

with_message_history=RunnableWithMessageHistory(model,get_session_history)

In [6]:
config={"configurable":{"session_id":"chat1"}}

In [7]:
response=with_message_history.invoke(
    [HumanMessage(content="Hi , My name is Krish and I am a Chief AI Engineer")],
    config=config
)

In [8]:
response.content

"Hello Krish, it's nice to meet you! That's a very impressive title.  \n\nWhat can I do for you today? Are you interested in discussing AI advancements, exploring potential applications, or perhaps brainstorming new ideas? I'm ready to listen and help in any way I can.\n"

In [9]:
with_message_history.invoke(
    [HumanMessage(content="What's my name?")],
    config=config,
)

AIMessage(content='Your name is Krish.  \n\nYou told me at the beginning of our conversation. 😊  How can I help you further? \n', response_metadata={'token_usage': {'completion_tokens': 30, 'prompt_tokens': 100, 'total_tokens': 130, 'completion_time': 0.054545455, 'prompt_time': 0.008262906, 'queue_time': 0.273242794, 'total_time': 0.062808361}, 'model_name': 'Gemma2-9b-It', 'system_fingerprint': 'fp_10c08bf97d', 'finish_reason': 'stop', 'logprobs': None}, id='run-519bd520-e3f1-45bf-b5a9-91667e2edf57-0', usage_metadata={'input_tokens': 100, 'output_tokens': 30, 'total_tokens': 130})

In [10]:
## change the config-->session id
config1={"configurable":{"session_id":"chat2"}}
response=with_message_history.invoke(
    [HumanMessage(content="Whats my name")],
    config=config1
)
response.content

"As an AI, I have no memory of past conversations and do not know your name. If you'd like to tell me, I'm happy to know! 😊\n"

In [11]:
response=with_message_history.invoke(
    [HumanMessage(content="Hey My name is John")],
    config=config1
)
response.content

"Hi John, it's nice to meet you!  What can I do for you today? 😄  \n\n"

In [12]:
response=with_message_history.invoke(
    [HumanMessage(content="Whats my name")],
    config=config1
)
response.content

'Your name is John!  I remember now. 😊  \n\nIs there anything else I can help you with, John?\n'

### Prompt templates
Prompt Templates help to turn raw user information into a format that the LLM can work with. In this case, the raw user input is just a message, which we are passing to the LLM. Let's now make that a bit more complicated. First, let's add in a system message with some custom instructions (but still taking messages as input). Next, we'll add in more input besides just the messages.

In [13]:
from langchain_core.prompts import ChatPromptTemplate,MessagesPlaceholder
prompt=ChatPromptTemplate.from_messages(
    [
        ("system","You are a helpful assistant.Answer all the question to the best of your ability"),
        MessagesPlaceholder(variable_name="messages")
    ]
)

chain=prompt|model

In [14]:
chain.invoke({"messages":[HumanMessage(content="Hi My name is Krish")]})

AIMessage(content="Hello Krish! It's nice to meet you.  \n\nWhat can I do for you today? I'm ready for any questions you have. 😊  \n\n", response_metadata={'token_usage': {'completion_tokens': 37, 'prompt_tokens': 30, 'total_tokens': 67, 'completion_time': 0.067272727, 'prompt_time': 0.002119196, 'queue_time': 0.245135033, 'total_time': 0.069391923}, 'model_name': 'Gemma2-9b-It', 'system_fingerprint': 'fp_10c08bf97d', 'finish_reason': 'stop', 'logprobs': None}, id='run-0966de33-e17a-41fe-bf1d-d86b461f6a0b-0', usage_metadata={'input_tokens': 30, 'output_tokens': 37, 'total_tokens': 67})

In [15]:
with_message_history=RunnableWithMessageHistory(chain,get_session_history)

In [16]:
config = {"configurable": {"session_id": "chat3"}}
response=with_message_history.invoke(
    [HumanMessage(content="Hi My name is Krish")],
    config=config
)

response

AIMessage(content="Hi Krish, it's nice to meet you! \n\nWhat can I help you with today? 😊  \n\nI'm ready for any questions you have.  \n\n", response_metadata={'token_usage': {'completion_tokens': 39, 'prompt_tokens': 30, 'total_tokens': 69, 'completion_time': 0.070909091, 'prompt_time': 0.002185255, 'queue_time': 0.246414364, 'total_time': 0.073094346}, 'model_name': 'Gemma2-9b-It', 'system_fingerprint': 'fp_10c08bf97d', 'finish_reason': 'stop', 'logprobs': None}, id='run-bc9a64fd-7fa5-4c67-afcf-4029bba9e4fc-0', usage_metadata={'input_tokens': 30, 'output_tokens': 39, 'total_tokens': 69})

In [17]:
response = with_message_history.invoke(
    [HumanMessage(content="What's my name?")],
    config=config,
)

response.content

'Your name is Krish!  I remember that from our introduction. 😊  \n\n\n\n'

In [18]:
## Add more complexity

prompt = ChatPromptTemplate.from_messages(
    [
        (
            "system",
            "You are a helpful assistant. Answer all questions to the best of your ability in {language}.",
        ),
        MessagesPlaceholder(variable_name="messages"),
    ]
)

chain = prompt | model

In [19]:
response=chain.invoke({"messages":[HumanMessage(content="Hi My name is Krish")],"language":"Hindi"})
response.content

'नमस्ते कृष! 👋  मुझे बहुत खुशी हो कि आप मुझसे बात कर रहे हैं। \n\nआप मुझे कैसे मदद कर सकता हूँ? 😊\n'

Let's now wrap this more complicated chain in a Message History class. This time, because there are multiple keys in the input, we need to specify the correct key to use to save the chat history.

In [20]:
with_message_history=RunnableWithMessageHistory(
    chain,
    get_session_history,
    input_messages_key="messages"
)

In [21]:
config = {"configurable": {"session_id": "chat4"}}
repsonse=with_message_history.invoke(
    {'messages': [HumanMessage(content="Hi,I am Krish")],"language":"Hindi"},
    config=config
)
repsonse.content

'नमस्ते कृष्ण, मैं आपकी मदद करने के लिए तैयार हूँ। आपके कोई सवाल हैं?  😊\n'

In [22]:
response = with_message_history.invoke(
    {"messages": [HumanMessage(content="whats my name?")], "language": "Hindi"},
    config=config,
)

In [23]:
response.content

'आपका नाम कृष्ण है।  \n'

### Managing the Conversation History
One important concept to understand when building chatbots is how to manage conversation history. If left unmanaged, the list of messages will grow unbounded and potentially overflow the context window of the LLM. Therefore, it is important to add a step that limits the size of the messages you are passing in.
'trim_messages' helper to reduce how many messages we're sending to the model. The trimmer allows us to specify how many tokens we want to keep, along with other parameters like if we want to always keep the system message and whether to allow partial messages

In [24]:
from langchain_core.messages import SystemMessage,trim_messages
trimmer=trim_messages(
    max_tokens=45,
    strategy="last",
    token_counter=model,
    include_system=True,
    allow_partial=False,
    start_on="human"
)
messages = [
    SystemMessage(content="you're a good assistant"),
    HumanMessage(content="hi! I'm bob"),
    AIMessage(content="hi!"),
    HumanMessage(content="I like vanilla ice cream"),
    AIMessage(content="nice"),
    HumanMessage(content="whats 2 + 2"),
    AIMessage(content="4"),
    HumanMessage(content="thanks"),
    AIMessage(content="no problem!"),
    HumanMessage(content="having fun?"),
    AIMessage(content="yes!"),
]
trimmer.invoke(messages)

e:\Udemy\Gen-AI\venv\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
e:\Udemy\Gen-AI\venv\Lib\site-packages\huggingface_hub\file_download.py:143: UserWarning: `huggingface_hub` cache-system uses symlinks by default to efficiently store duplicated files but your machine does not support them in C:\Users\LENOVO\.cache\huggingface\hub\models--gpt2. Caching files will still work but in a degraded version that might require more space on your disk. This warning can be disabled by setting the `HF_HUB_DISABLE_SYMLINKS_WARNING` environment variable. For more details, see https://huggingface.co/docs/huggingface_hub/how-to-cache#limitations.
To support symlinks on Windows, you either need to activate Developer Mode or to run Python as an administrator. In order to activate developer mode, see this article: https://docs.micr

[SystemMessage(content="you're a good assistant"),
 HumanMessage(content='I like vanilla ice cream'),
 AIMessage(content='nice'),
 HumanMessage(content='whats 2 + 2'),
 AIMessage(content='4'),
 HumanMessage(content='thanks'),
 AIMessage(content='no problem!'),
 HumanMessage(content='having fun?'),
 AIMessage(content='yes!')]

In [25]:
from operator import itemgetter

from langchain_core.runnables import RunnablePassthrough

chain=(
    RunnablePassthrough.assign(messages=itemgetter("messages")|trimmer)
    | prompt
    | model
    
)

response=chain.invoke(
    {
    "messages":messages + [HumanMessage(content="What ice cream do i like")],
    "language":"English"
    }
)
response.content

"As an AI, I don't have access to your personal information, including your ice cream preferences. \n\nWhat's your favorite flavor? 😊🍦\n"

In [26]:
response = chain.invoke(
    {
        "messages": messages + [HumanMessage(content="what math problem did i ask")],
        "language": "English",
    }
)
response.content

'You asked "What is 2 + 2?".  😄'

In [27]:
## Lets wrap this in the MEssage History
with_message_history = RunnableWithMessageHistory(
    chain,
    get_session_history,
    input_messages_key="messages",
)
config={"configurable":{"session_id":"chat5"}}

In [28]:
response = with_message_history.invoke(
    {
        "messages": messages + [HumanMessage(content="whats my name?")],
        "language": "English",
    },
    config=config,
)

response.content

"As an AI, I don't have access to any past conversations or personal information about you, so I don't know your name. 😊 \n\nWould you like to tell me?\n"

In [29]:
response = with_message_history.invoke(
    {
        "messages": [HumanMessage(content="what math problem did i ask?")],
        "language": "English",
    },
    config=config,
)

response.content

"As a large language model, I have no memory of past conversations. So I don't know what math problem you asked. \n\nIf you'd like to ask me a math problem, I'm happy to help! 😊 \n\n"